In [13]:
import pandas as pd
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4.element
import warnings
warnings.filterwarnings('ignore')
import re
import time
from datetime import datetime

In [14]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [15]:
def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [16]:
folder = '비행체부품'

In [17]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\2. DOWNLOAD\\{}'.format(folder))

df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\2. DOWNLOAD\\{}\\'.format(folder)+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

df_all

,date,title,linkcontent,link,content
0,2023-07-03 10:17:03,"NST, '에어모빌리티 소재' 등 4개 융합연구단 출범",NaN,https://n.news.naver.com/mnews/article/018/000...,"[\n비행체 친환경 소재, 에너지통합기술 개발 시작[이데일리 강민구 기자] 국내 정..."
1,2023-07-08 14:31:03,도심 위 하늘길 열어라…통신사들이 UAM 사업 눈독들이는 이유,NaN,https://n.news.naver.com/mnews/article/003/001...,"[\nSKT, 글로벌 기체 제조사 조비와 협력…KT, 현대차·현대건설과 컨소LGU+..."
2,2023-05-24 09:42:01,‘국내 기술로 제작’…파브(PAV?개인항공비행체) ‘시험비행’ 성공,NaN,https://n.news.naver.com/mnews/article/011/000...,"[\n㈜숨비, “국방과학연구소(ADD)의 현지 평가결과?성공 판정”\n\n\n\n항..."
3,2022-05-02 13:42:02,전자약·항공우주부품·배터리·무인비행체…경남 스마트특성화 4개 선정,NaN,https://n.news.naver.com/mnews/article/079/000...,"[\n핵심요약산업통상자원부 스마트특성화 기반 구축사업, 역대 최다 4개 사업 선정\..."
4,2023-05-26 16:07:24,국내 기술로 제작한 '파브·개인비행체' 시험비행 성공…2030년 상용화,NaN,https://n.news.naver.com/mnews/article/421/000...,"[\n국방과학연구원, 숨비의 파브 시험비행 '성공' 판정\n\n\n\n(주)숨비가 ..."
...,...,...,...,...,...
18,2022-10-10 08:03:01,[박상욱의 기후 1.5] 세상에 착한 에너지는 없다?,NaN,https://n.news.naver.com/mnews/article/437/000...,[\n'먼 미래'에서 '내 일'로 찾아온 기후변화 (152)'누가 더 나쁜가' 겨루...
19,2022-10-11 13:09:01,"탄소산업진흥원, 오는 26일부터 '카본코리아 2022' 개최",NaN,https://n.news.naver.com/mnews/article/030/000...,[\n\n\n\n\n카본코리아 2022. &lt;자료 한국탄소산업진흥원 제공&gt;...
20,2022-10-01 05:01:01,[석민의News픽] MBC '방송 자막 조작'은 언론자유가 아니라 범죄다!,NaN,https://n.news.naver.com/mnews/article/088/000...,"[\n▷윤(尹)의 고독한 전쟁, 대한민국 바로세우기…MBC·이재명의 민주당, 운명은..."
21,2022-10-16 18:02:04,[ET단상]국가 안보의 비밀병기 EMP 측정기술,NaN,https://n.news.naver.com/mnews/article/030/000...,[\n\n\n\n\n이동준 한국표준과학연구원 전자파표준그룹 책임연구원진주만 공습 당...


In [18]:
df_all.drop('link', axis=1, inplace=True)
# df_all.drop('Unnamed: 0', axis=1, inplace=True)
df_all.drop('linkcontent', axis=1, inplace=True)

In [19]:
df_all.dropna(axis=0, inplace=True)
df_all

,date,title,content
0,2023-07-03 10:17:03,"NST, '에어모빌리티 소재' 등 4개 융합연구단 출범","[\n비행체 친환경 소재, 에너지통합기술 개발 시작[이데일리 강민구 기자] 국내 정..."
1,2023-07-08 14:31:03,도심 위 하늘길 열어라…통신사들이 UAM 사업 눈독들이는 이유,"[\nSKT, 글로벌 기체 제조사 조비와 협력…KT, 현대차·현대건설과 컨소LGU+..."
2,2023-05-24 09:42:01,‘국내 기술로 제작’…파브(PAV?개인항공비행체) ‘시험비행’ 성공,"[\n㈜숨비, “국방과학연구소(ADD)의 현지 평가결과?성공 판정”\n\n\n\n항..."
3,2022-05-02 13:42:02,전자약·항공우주부품·배터리·무인비행체…경남 스마트특성화 4개 선정,"[\n핵심요약산업통상자원부 스마트특성화 기반 구축사업, 역대 최다 4개 사업 선정\..."
4,2023-05-26 16:07:24,국내 기술로 제작한 '파브·개인비행체' 시험비행 성공…2030년 상용화,"[\n국방과학연구원, 숨비의 파브 시험비행 '성공' 판정\n\n\n\n(주)숨비가 ..."
...,...,...,...
18,2022-10-10 08:03:01,[박상욱의 기후 1.5] 세상에 착한 에너지는 없다?,[\n'먼 미래'에서 '내 일'로 찾아온 기후변화 (152)'누가 더 나쁜가' 겨루...
19,2022-10-11 13:09:01,"탄소산업진흥원, 오는 26일부터 '카본코리아 2022' 개최",[\n\n\n\n\n카본코리아 2022. &lt;자료 한국탄소산업진흥원 제공&gt;...
20,2022-10-01 05:01:01,[석민의News픽] MBC '방송 자막 조작'은 언론자유가 아니라 범죄다!,"[\n▷윤(尹)의 고독한 전쟁, 대한민국 바로세우기…MBC·이재명의 민주당, 운명은..."
21,2022-10-16 18:02:04,[ET단상]국가 안보의 비밀병기 EMP 측정기술,[\n\n\n\n\n이동준 한국표준과학연구원 전자파표준그룹 책임연구원진주만 공습 당...


In [20]:
df_all['content'] = df_all['content'].apply(text_clean)
df_all['content'] = df_all['content'].apply(clean_text)
df_all

,date,title,content
0,2023-07-03 10:17:03,"NST, '에어모빌리티 소재' 등 4개 융합연구단 출범",비행체 친환경 소재 에너지통합기술 개발 시작이데일리 강민구 기자 국내 정부출연연구기...
1,2023-07-08 14:31:03,도심 위 하늘길 열어라…통신사들이 UAM 사업 눈독들이는 이유,글로벌 기체 제조사 조비와 협력 현대차현대건설과 컨소 서울교통공사와 복합환승센터 ...
2,2023-05-24 09:42:01,‘국내 기술로 제작’…파브(PAV?개인항공비행체) ‘시험비행’ 성공,숨비 국방과학연구소의 현지 평가결과성공 판정항공테크기업인 숨비는 최근 한국국방과학연...
3,2022-05-02 13:42:02,전자약·항공우주부품·배터리·무인비행체…경남 스마트특성화 4개 선정,
4,2023-05-26 16:07:24,국내 기술로 제작한 '파브·개인비행체' 시험비행 성공…2030년 상용화,강남주 기자 국내 기술로 개발된 개인항공비행체가 시험비행에 성공하면서 파브 상용화...
...,...,...,...
18,2022-10-10 08:03:01,[박상욱의 기후 1.5] 세상에 착한 에너지는 없다?,먼 미래에서 내 일로 찾아온 기후변화 누가 더 나쁜가 겨루는 자리가 되어버린 국정감...
19,2022-10-11 13:09:01,"탄소산업진흥원, 오는 26일부터 '카본코리아 2022' 개최",카본코리아 자료 한국탄소산업진흥원 제공탄소 소재 및 융복합 산업 생태계를 조성하고 ...
20,2022-10-01 05:01:01,[석민의News픽] MBC '방송 자막 조작'은 언론자유가 아니라 범죄다!,이화영 정진상 그 다음 이재명당황한 민주당의 무리수 승부수수원지검 형사부부장 김영...
21,2022-10-16 18:02:04,[ET단상]국가 안보의 비밀병기 EMP 측정기술,이동준 한국표준과학연구원 전자파표준그룹 책임연구원진주만 공습 당시 미국군은 일본군 ...


In [21]:
df_all['title'] = df_all['title'].apply(text_clean)
df_all['title'] = df_all['title'].apply(clean_text)
df_all

,date,title,content
0,2023-07-03 10:17:03,에어모빌리티 소재 등 개 융합연구단 출범,비행체 친환경 소재 에너지통합기술 개발 시작이데일리 강민구 기자 국내 정부출연연구기...
1,2023-07-08 14:31:03,도심 위 하늘길 열어라통신사들이 사업 눈독들이는 이유,글로벌 기체 제조사 조비와 협력 현대차현대건설과 컨소 서울교통공사와 복합환승센터 ...
2,2023-05-24 09:42:01,국내 기술로 제작파브개인항공비행체 시험비행 성공,숨비 국방과학연구소의 현지 평가결과성공 판정항공테크기업인 숨비는 최근 한국국방과학연...
3,2022-05-02 13:42:02,전자약항공우주부품배터리무인비행체경남 스마트특성화 개 선정,
4,2023-05-26 16:07:24,국내 기술로 제작한 파브개인비행체 시험비행 성공년 상용화,강남주 기자 국내 기술로 개발된 개인항공비행체가 시험비행에 성공하면서 파브 상용화...
...,...,...,...
18,2022-10-10 08:03:01,박상욱의 기후 세상에 착한 에너지는 없다,먼 미래에서 내 일로 찾아온 기후변화 누가 더 나쁜가 겨루는 자리가 되어버린 국정감...
19,2022-10-11 13:09:01,탄소산업진흥원 오는 일부터 카본코리아 개최,카본코리아 자료 한국탄소산업진흥원 제공탄소 소재 및 융복합 산업 생태계를 조성하고 ...
20,2022-10-01 05:01:01,석민의픽 방송 자막 조작은 언론자유가 아니라 범죄다,이화영 정진상 그 다음 이재명당황한 민주당의 무리수 승부수수원지검 형사부부장 김영...
21,2022-10-16 18:02:04,단상국가 안보의 비밀병기 측정기술,이동준 한국표준과학연구원 전자파표준그룹 책임연구원진주만 공습 당시 미국군은 일본군 ...


In [22]:
df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\{}_merge.csv'.format(folder), index=False)

In [23]:
NewsDate = df_all['date'].to_list()
NewsTitle = df_all['title'].to_list()
NewsContent = df_all['content'].to_list()

In [24]:
for date,title,content in zip(NewsDate, NewsTitle, NewsContent):
    sql = "INSERT INTO past_news_aircraftparts(NewsDates, NewsTitles, NewsContents) VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
    cur.execute(sql)
conn.commit()
conn.close()